<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Sntence_entailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**work in progress**

# Data Exploration

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!unzip train.zip

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train_filtered  = train.filter(['premise','hypothesis','label'])

In [ ]:
train_filtered.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [ ]:
#get cell value
train.iloc[1]['premise']
train.iloc[1]['hypothesis']

In [ ]:
dataset = nlp.Dataset.from_pandas(train_filtered)
ds_splitted = dataset.train_test_split(test_size=0.1)

In [ ]:
ds_splitted['train']['premise'][1]

'But in all probability the girl will have entirely forgotten the intervening period, and will take up life where she left off at the sinking of the Lusitania."'

# Model

In [3]:
%%capture
!pip install pytorch_lightning
!pip install transformers
!pip install nlp

In [4]:
import torch as th
import pytorch_lightning as pl
import nlp
import transformers

In [5]:
from transformers import BertTokenizer

In [ ]:
#example of tokenizing 2 sentences into
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = ds_splitted['train']['premise'][1]
sequence_b = ds_splitted['train']['hypothesis'][1]
encoded_dict = tokenizer(sequence_a, sequence_b,                     
                         max_length=128, 
                         truncation = True,
                         pad_to_max_length=True,
                         return_tensors = 'pt')
#decoded = tokenizer.decode(encoded_dict["input_ids"])
print(type(encoded_dict))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
outputs = model(**encoded_dict)

In [ ]:
outputs[1].shape

torch.Size([1, 768])

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs[0]

In [ ]:
m = th.nn.ReLU()
type(m)

torch.nn.modules.activation.ReLU

In [6]:
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.brt = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer1 = th.nn.Linear(768,3)
        #self.relu = th.nn.ReLU()
        #self.layer2 = th.nn.Linear(32,3)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

        def _tokenize(x):
            print('in tokenize')
            return tokenizer(
                    x['premise'],
                    x['hypothesis'],
                    max_length=64, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            df = pd.read_csv('train.csv', nrows=1000)
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = nlp.Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)

            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            return ds_flt['train'], ds_flt['test']

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks, token_type_ids):
        logits = self.brt(input_ids, masks, token_type_ids)
        out = self.layer1(logits[1])
        #out = self.layer1(logits[0][:,0,:])
        #out = self.relu(out)
        #out = self.layer2(out)
        return out

    def training_step(self, batch, batch_idx):
        #print('training step function')
        logits = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label']).mean()
        training_loss = {'train_loss': loss}
        print('in  training_step - train_loss {}'.format(training_loss))
        return {'loss': loss, 'log': training_loss} 

    def validation_step(self, batch, batch_idx):
        #print('validation step function')
        logits = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        #print('in  validation_step logits shape - {}'.format(logits))
        #print('in  validation_step logits argmax(1) - {}'.format(logits.argmax(dim=1)))
        #print('in  validation_step logits argmax(-1) - {}'.format(logits.argmax(-1)))
        #print('in  validation_step label  - {}'.format(batch['label']))

        loss = self.loss(logits, batch['label'])
        #print('in validation_stept - loss - {}'.format(loss))
        acc = (logits.argmax(-1) == batch['label']).float()
        #print('in validation_stept - acc - {}'.format(acc))
        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        print('in validation step: out: {}'.format(out)) 
        return {**out, 'log': out}

    def train_dataloader(self):
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=8,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=8,
                drop_last=False,
                shuffle=True,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [ ]:
#Example of Softmax calculation in Pytorch
inp = th.Tensor([0.0584,-0.1041,0.1367]); inp
probs = th.nn.Softmax(dim=-1); probs
res = probs(inp); res

tensor([0.3411, 0.2900, 0.3689])

In [14]:
loss = th.Tensor([0.0584,-0.1041,0.1367])
result = pl.TrainResult()
result.log('train_loss', loss)
print(result)

{'train_loss': tensor([ 0.0584, -0.1041,  0.1367])}


In [24]:
!rm -rf ./logs/
def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=5,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs/', name='entail'),
    )
    trainer.fit(model)

#!rm -rf ./logs/

start_time = time.time()
model_fit()
execution_time = time.time() - start_time

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



in tokenize


in tokenize

in tokenize


in tokenize



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name   | Type             | Params
--------------------------------------------
0 | brt    | BertModel        | 109 M 
1 | layer1 | Linear           | 2 K   
2 | loss   | CrossEntropyLoss | 0     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


...

in validation step: out: {'val_loss': tensor(1.3209, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}
in  training_step - train_loss {'train_loss': tensor(1.5240, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.0611, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(0.8621, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.3585, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.3162, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.2682, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(0.9772, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.1372, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(

in validation step: out: {'val_loss': tensor(1.1003, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}
in  training_step - train_loss {'train_loss': tensor(1.1640, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.0091, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(0.8691, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(0.9868, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.1221, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.4303, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(2.0452, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.6309, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(

in validation step: out: {'val_loss': tensor(1.1246, device='cuda:0'), 'val_acc': tensor(0.3200, device='cuda:0')}
in  training_step - train_loss {'train_loss': tensor(1.1523, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.0309, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.2906, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.2997, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.1778, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.0497, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.4134, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(1.1451, device='cuda:0', grad_fn=<MeanBackward0>)}
in  training_step - train_loss {'train_loss': tensor(

in validation step: out: {'val_loss': tensor(1.1307, device='cuda:0'), 'val_acc': tensor(0.3500, device='cuda:0')}


Saving latest checkpoint..


In [23]:
print('training execution time - {}'.format(execution_time))

training execution time - 91.34599828720093


In [11]:
#%reload_ext tensorboard

In [25]:
%load_ext tensorboard
%tensorboard --logdir logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 280), started 1:38:17 ago. (Use '!kill 280' to kill it.)

<IPython.core.display.Javascript object>

# Sandbox